### Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math as math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers import Dropout
from keras.layers import *
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from joblib import parallel_backend

from time import time
from tensorflow.keras.callbacks import TensorBoard
import warnings
warnings.filterwarnings('ignore')

### Load data

In [2]:
VIX = pd.read_fwf('VIX.txt', names=['Year', 'Clos'])

In [3]:
VIX.head()

,Year,Clos
0,06/01/1995,"13,13"
1,13/01/1995,"11,1"
2,20/01/1995,"12,15"
3,27/01/1995,"11,25"
4,03/02/1995,"10,98"


In [4]:
data = pd.read_fwf('SPX.txt', names=['Year', 'Clos'])

In [5]:
data.head()

,Year,Clos
0,06/01/1995,"460,68"
1,13/01/1995,"465,97"
2,20/01/1995,"464,78"
3,27/01/1995,"470,39"
4,03/02/1995,"478,64"


## Data préparation

> Using onlye the Clos column

In [8]:
data.rename(columns={'Year': 'Date'}, inplace=True)

In [9]:
data['Date'] = pd.to_datetime(data['Date'], format="%d/%m/%Y")

In [10]:
data.head()

,Date,Clos
0,1995-01-06,"460,68"
1,1995-01-13,"465,97"
2,1995-01-20,"464,78"
3,1995-01-27,"470,39"
4,1995-02-03,"478,64"


In [9]:
data['Clos'] = data['Clos'].apply(lambda x: x.replace(',', '.'))
data['Clos']= data['Clos'].astype(float)

In [12]:
data['Year']=pd.DatetimeIndex(data['Date']).year

In [13]:
data['Week']=pd.DatetimeIndex(data['Date']).week

>

In [45]:
train_data_tmp = data[data['Year']<interval_data]

TypeError: '(slice(0, 887, None), slice(None, None, None))' is an invalid key

In [14]:
train = np.array(train_data_tmp['Clos'])

train = train.reshape((len(train_data_tmp), 1))

In [15]:
train_data = scaler.fit_transform(train)

In [19]:
x_train = []
y_train = []

for x in range(180, len(train_data)):
    x_train.append(train_data[x-180:x, 0])
    y_train.append(train_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

train_data = data[data.Year < 2012]
test_data = data[data.Year >= 2012]

x_train = train_data[['Week', 'Year']]
y_train = train_data.Clos

x_test = test_data[['Week', 'Year']]
y_test = test_data.Clos

In [11]:
X = np.array(data['Clos'])
X = X.reshape((1292, 1))

In [12]:
training_data_len = math.ceil(len(X)* 0.75) # training set has 75% of the data
training_data_len

969

In [13]:
sc = MinMaxScaler(feature_range=(0,1))
training_data = sc.fit_transform(X)
training_data

array([[0.        ],
       [0.00206228],
       [0.00159837],
       ...,
       [0.98682323],
       [0.97501092],
       [0.97122942]])

In [14]:
split_val = 887
train_data = training_data[0:split_val  , : ]

X_train = []
y_train = []
for i in range(180, len(train_data)):
    X_train.append(train_data[i-180:i, 0])
    y_train.append(train_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train) # converting into numpy sequences to train the LSTM model
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print('Number of rows and columns: ', X_train.shape)  #(854 values, 180 time-steps, 1 output)


Number of rows and columns:  (707, 180, 1)


In [15]:
test_data = training_data[split_val - 180: , : ]

# Create the x_test and y_test data sets
X_test = []
y_test =  X[split_val : , : ]
for i in range(180,len(test_data)):
    X_test.append(test_data[i-180:i,0])

# Convert x_test to a numpy array
X_test = np.array(X_test)

#Reshape the data into the shape accepted by the LSTM
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
print('Number of rows and columns: ', X_test.shape)


Number of rows and columns:  (405, 180, 1)


<H1 style="color: red; text-align:center"> Models </H1>

## LSTM

In [19]:
x_train.shape

(887, 2)

In [20]:
model = Sequential()

In [21]:
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))

In [22]:
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 50))
model.add(Dropout(0.3))

model.add(Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [23]:
History = model.fit(x_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100


KeyboardInterrupt: 

***
#### Test

### Random Reg

In [16]:
from sklearn.ensemble import RandomForestRegressor

In [17]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [18]:
rf = RandomForestRegressor()

In [22]:
rf = rf.fit(X_train, y_train)

ValueError: Found array with dim 3. RandomForestRegressor expected <= 2.

In [41]:
predict = rf.predict(x_test)

In [44]:
r2_score(y_test, predict)

-3.2035394204226284

In [45]:
accuracy_score(y_test, predict)

ValueError: continuous is not supported

### Linear Reg

In [24]:
from sklearn.linear_model import LinearRegression

In [28]:
predict = model_linear.fit(x_train, y_train)

In [32]:
predict.score(x_test, y_test)

-1.891444000378394

In [31]:
model_linear = LinearRegression()

model_linear.fit(x_train, y_train) 
model_linear.score(x_test, y_test)

-1.891444000378394

In [142]:
test_data_tmp = data[data['Year']>interval_data]

In [143]:
test = np.array(test_data_tmp['Clos'])

test = test.reshape((len(test_data_tmp), 1))

In [144]:
test_data = scaler.fit_transform(test)

In [145]:
len(test_data)

405

In [146]:
X_data = np.array(data['Clos'])
X_data = X_data.reshape((1292, 1))

In [162]:
test_data

array([[0.00000000e+00],
       [6.40732265e-03],
       [2.14530892e-02],
       [2.20251716e-02],
       [3.83867277e-02],
       [3.70709382e-02],
       [4.77116705e-02],
       [5.02860412e-02],
       [5.25171625e-02],
       [5.32036613e-02],
       [7.22540046e-02],
       [6.82494279e-02],
       [7.47139588e-02],
       [6.87643021e-02],
       [5.28604119e-02],
       [5.76086957e-02],
       [7.17963387e-02],
       [5.22311213e-02],
       [4.31922197e-02],
       [9.95423341e-03],
       [2.28832952e-02],
       [1.14416476e-04],
       [2.73455378e-02],
       [3.71853547e-02],
       [3.27231121e-02],
       [4.82265446e-02],
       [4.39359268e-02],
       [4.51372998e-02],
       [4.85125858e-02],
       [6.18421053e-02],
       [6.47025172e-02],
       [7.32265446e-02],
       [8.02631579e-02],
       [7.62585812e-02],
       [7.36270023e-02],
       [9.15903890e-02],
       [1.07494279e-01],
       [1.04290618e-01],
       [9.31350114e-02],
       [1.04748284e-01],


In [206]:
x_test = []
#y_test = test_data[100:,:]
y_test = []
for x in range(100, len(test_data)):
    x_test.append(test_data[x-100:x, 0])
    y_test.append(test_data[x, 0])

x_test = np.array(x_test)
y_test = np.array(y_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = np.reshape(y_test, (y_test.shape[0],1))

In [207]:
y_test.shape

(305, 1)

In [208]:
x_test.shape

(305, 100, 1)